以一个电商的例子记录如何从0到1建立用户画像。企业在运营过程中积累了大量经营数据和用户行为数据，建立企业的用户画像简单来说就是给企业每个用户打上各种“标签”，利用这个标签系统就能有效地进行精细化运营。
这里利用Python，SQL等技术手段整合业务数据、埋点数据和日志数据等，并建立用户画像的数据仓库。利用数据仓库进行经营分析，营销优化，以及个性化推荐等服务。

# 目录

1. 项目目标与数据收集
2. 数据指标体系设计
3. 用户画像数据仓库开发
4. 用户画像的应用
5. 总结